In [2]:
import pandas as pd
import re

# generate dictionary
for every html file, extract the mz value, assign the lipid sum composition, class, timepoint, path to lcms, path to pieplot

In [3]:
import os

In [13]:

# Load the LCMS and PIECHART filenames from CSV files
lcms_file_path = 'filenamesLCMS.csv'  # Replace with the actual path to filenamesLCMS.csv
piechart_file_path = 'filenamesPIECHART.csv'  # Replace with the actual path to filenamesPIECHART.csv

# Read the LCMS and PIECHART filenames from the CSV files
lcms_df = pd.read_csv(lcms_file_path, names=['filename'], skiprows=1)

piechart_df = pd.read_csv(piechart_file_path, header=None, names=['filename'])
umap_df = pd.DataFrame(os.listdir('UMAP_PLOTS/'), columns=['filename'])

# Extract numeric values from filenames for matching
lcms_df['key'] = lcms_df['filename'].apply(lambda x: re.findall(r'\d+\.\d+', x))
lcms_df['key'] = lcms_df['key'].apply(lambda x: x[0] if x else None)
lcms_df['key'] = lcms_df['key'].astype(float)

piechart_df['key'] = piechart_df['filename'].apply(lambda x: re.findall(r'\d+\.\d+', x))
piechart_df['key'] = piechart_df['key'].apply(lambda x: x[0] if x else None)
piechart_df['key'] = piechart_df['key'].astype(float)
piechart_df = piechart_df.iloc[1:]

umap_df['key'] = umap_df['filename'].apply(lambda x: re.findall(r'\d+\.\d+', x))
umap_df['key'] = umap_df['key'].apply(lambda x: x[0] if x else None)
umap_df['key'] = umap_df['key'].astype(float)


In [15]:
import os
import numpy as np

In [16]:
# load annotation file
df_annotation = pd.read_csv('valid_mz_rev_result_df_amount_rounded3_lipid_call.csv')

In [17]:
df_annotation.columns

Index(['Estimated_mz', 'Lipid_sum_composition',
       'Most_abundant_lipid_sum_composition', 'Most_abundant_lipid_specie',
       'Lipid_sum_composition_amount', 'Named_lipid', 'Adduct',
       'Theoretical_mz_with_selected_adduct', 'Formula', 'Neutral_mass',
       'Delta_mz', 'Lipid_specie_from_lcms', 'Average_pmol_ng_72_hpf_total',
       'Lipids_species', 'Retention Time (min)', 'RT Window (min)',
       'Precursor (m/z)', 'Product (m/z)', 'Collision Energy (V)',
       'RF Lens (V)', 'Ionisation', 'formula', 'adduct', 'ion', 'mz', 'fdr',
       'Sum_composition', 'moleculeNames', 'moleculeIds', 'Database',
       'Same_found_in_metaspace'],
      dtype='object')

In [18]:
df_annotation = df_annotation[['Most_abundant_lipid_sum_composition', 'Theoretical_mz_with_selected_adduct']]
df_annotation['Class'] = df_annotation.Most_abundant_lipid_sum_composition.apply(lambda x: x.split(' ')[0])
df_annotation.Theoretical_mz_with_selected_adduct = df_annotation.Theoretical_mz_with_selected_adduct.apply(lambda x: float(str(x).split('; ')[0]))

In [20]:
def find_matching_lcms_image(key, lcms_df):
    """Find the closest LCMS image filename for a given key."""
    if key is None:
        return None
    ix = np.argmin(np.abs(lcms_df['key'].values - key))
    closest_match = lcms_df.iloc[ix]
    return f'data_lcms/{closest_match["filename"]}'

def find_matching_umap_image(key, umap_df):
    """Find the closest LCMS image filename for a given key."""
    if key is None:
        return None
    ix = np.argmin(np.abs(umap_df['key'].values - key))
    closest_match = umap_df.iloc[ix]
    return f'UMAP_PLOTS/{closest_match["filename"]}'

def find_matching_piechart_image(key, piechart_df):
    """Find the closest PIECHART image filename for a given key."""
    if key is None:
        return None
    ix = np.argmin(np.abs(piechart_df['key'].values - key))
    closest_match = piechart_df.iloc[ix]
    # print(closest_match)
    return f'PIE_PLOTS/{closest_match["filename"]}.png'



In [21]:
# fill the 72hpf first
entries = []

for entry in os.listdir('data_html/72hpf'):
    lipid_dict = {}
    mz_query = float(entry.split('.')[0].replace('_', '.'))

    # identify the lipid
    ix = np.argmin(np.abs(mz_query - df_annotation.Theoretical_mz_with_selected_adduct))
    
    
    lipid_dict['name'] = df_annotation.iloc[ix].Most_abundant_lipid_sum_composition
    lipid_dict['file'] = os.path.join('data_html/72hpf', entry)
    lipid_dict['class'] = df_annotation.iloc[ix].Class
    lipid_dict['timepoint'] = '72 hpf'

    # identify the file from lcms
    lcms_path = find_matching_lcms_image(mz_query, lcms_df)
    lipid_dict['image'] = lcms_path

    # identify the file from piecharts
    piechart_path = find_matching_piechart_image(mz_query, piechart_df)
    lipid_dict['additionalImage'] = piechart_path


    # umaps
    # identify the file from piecharts
    umap_path = find_matching_umap_image(mz_query, umap_df)
    lipid_dict['tertiaryImage'] = umap_path
    
    entries.append(lipid_dict)


In [22]:
entries

[{'name': 'LPC 20:5',
  'file': 'data_html/72hpf/523_359.html',
  'class': 'LPC',
  'timepoint': '72 hpf',
  'image': 'data_lcms/523.3506.png',
  'additionalImage': 'PIE_PLOTS/522.3554.png',
  'tertiaryImage': 'UMAP_PLOTS/522.355.png'},
 {'name': 'PC O-40:8',
  'file': 'data_html/72hpf/854_553.html',
  'class': 'PC',
  'timepoint': '72 hpf',
  'image': 'data_lcms/854.546.png',
  'additionalImage': 'PIE_PLOTS/853.7255.png',
  'tertiaryImage': 'UMAP_PLOTS/854.566.png'},
 {'name': 'PI 33:0',
  'file': 'data_html/72hpf/807_543.html',
  'class': 'PI',
  'timepoint': '72 hpf',
  'image': 'data_lcms/807.5376.png',
  'additionalImage': 'PIE_PLOTS/806.5694.png',
  'tertiaryImage': 'UMAP_PLOTS/806.568.png'},
 {'name': 'PE 40:9',
  'file': 'data_html/72hpf/808_486.html',
  'class': 'PE',
  'timepoint': '72 hpf',
  'image': 'data_lcms/808.4888.png',
  'additionalImage': 'PIE_PLOTS/808.5481.png',
  'tertiaryImage': 'UMAP_PLOTS/808.569.png'},
 {'name': 'LPE 22:1',
  'file': 'data_html/72hpf/536_371.

In [57]:
out

'data_lcms/523.3506.png'

In [53]:
lcms_df.sort_values('filename')

,filename,key
32,471.3193.png,471.3193
33,476.313.png,476.3130
34,478.3286.png,478.3286
35,480.3448.png,480.3448
36,492.3443.png,492.3443
...,...,...
299,909.5463.png,909.5463
300,923.562.png,923.5620
301,924.5596.png,924.5596
302,927.7412.png,927.7412


In [48]:
lipid_dict

{'name': 'LPC 20:5', 'file': 'data_html/72hpf/523_359.html', 'class': 'LPC'}

In [43]:
mz_query

523.359

In [40]:
mz_query - df_annotation.Theoretical_mz_with_selected_adduct

0       52.0397
1       47.0460
2       45.0304
3       43.0142
4       31.0147
         ...   
276   -386.1873
277   -400.2030
278   -401.2006
279   -404.3822
280   -433.2632
Name: Theoretical_mz_with_selected_adduct, Length: 281, dtype: float64

In [5]:



def extract_key_from_file_path(filepath):
    """Extracts the key from the file path (format: nnn_mmm.html) to match against LCMS and PIECHART keys."""
    
    return float(filepath.split('/')[-1].split('.html')[0].replace('_', '.'))





# Process entries to add image and additionalImage paths
processed_entries = []

for entry in entries:
    # Extract the key from the entry file path
    key = extract_key_from_file_path(entry['file'])
    
    # Find matching LCMS and PIECHART images
    image_path = find_matching_lcms_image(key, lcms_df)
    additional_image_path = find_matching_piechart_image(key, piechart_df)
    
    # Add the image paths to the entry
    entry['image'] = image_path if image_path else ''
    entry['additionalImage'] = additional_image_path if additional_image_path else ''
    
    processed_entries.append(entry)

# Display the final processed entries
for entry in processed_entries:
    print(entry)

NameError: name 'name' is not defined

0      754.5357
1      755.5334
2      758.5694
3      759.5646
4      760.4888
         ...   
299    909.5463
300    923.5620
301    924.5596
302    927.7412
303    956.6222
Name: key, Length: 304, dtype: float64

In [9]:
keypiechart_df['filename'].apply(lambda x: re.findall(r'\d+\.\d+', x))

492.345

In [10]:
lcms_df

,filename,key
NaN,0,None
0.0,754.5357.png,754.5357
1.0,755.5334.png,755.5334
2.0,758.5694.png,758.5694
3.0,759.5646.png,759.5646
...,...,...
299.0,909.5463.png,909.5463
300.0,923.562.png,923.562
301.0,924.5596.png,924.5596
302.0,927.7412.png,927.7412
